# Earth System Simulation - Google Colab Version

This notebook shows how to run the Earth system simulation in Google Colab.

## 1. Setup Environment

In [ ]:
# Install required packages
!pip install torch numpy matplotlib pyyaml tqdm plotly

In [ ]:
# Clone repository
!git clone https://github.com/yourusername/earth_system_sim.git
%cd earth_system_sim

In [ ]:
# Set up Python path
import os
import sys
from pathlib import Path

# Get absolute paths
project_root = Path.cwd().absolute()
src_path = project_root / 'src'
scripts_path = project_root / 'scripts'

# Add paths to sys.path
paths_to_add = [str(project_root), str(src_path), str(scripts_path)]
for path in paths_to_add:
    if path not in sys.path:
        sys.path.insert(0, path)

print("Python path:")
for p in sys.path:
    print(f"  {p}")

In [ ]:
# Create __init__.py files if they don't exist
def ensure_init_files():
    """Create necessary __init__.py files."""
    init_paths = [
        'src/__init__.py',
        'src/models/__init__.py',
        'src/models/physical/__init__.py',
        'src/models/biosphere/__init__.py',
        'src/models/geosphere/__init__.py',
        'src/integration/__init__.py',
        'src/visualization/__init__.py'
    ]
    
    for init_path in init_paths:
        path = project_root / init_path
        if not path.exists():
            path.parent.mkdir(parents=True, exist_ok=True)
            path.touch()
            print(f"Created: {init_path}")

ensure_init_files()

In [ ]:
# Import dependencies
import torch
import numpy as np
import matplotlib.pyplot as plt
import yaml
from tqdm.notebook import tqdm
from typing import Dict, List, Tuple, Optional, Union, Any

# Explicitly import from scripts
from run_simulation import EarthSystemSimulation

def verify_system():
    """Verify system configuration."""
    print("System Configuration:")
    print(f"Python version: {sys.version}")
    print(f"PyTorch version: {torch.__version__}")
    print(f"NumPy version: {np.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA device: {torch.cuda.get_device_name(0)}")

# Run system verification
verify_system()

## 2. Load and Validate Configuration

In [ ]:
def validate_config(config):
    """Validate configuration structure and values."""
    required_keys = ['grid_height', 'grid_width', 'physical_system', 
                     'biosphere', 'geosphere', 'integration']
    
    # Check required keys
    for key in required_keys:
        if key not in config:
            print(f"Missing required key: {key}")
            return False
    
    # Validate grid dimensions
    if config['grid_height'] <= 0 or config['grid_width'] <= 0:
        print("Invalid grid dimensions")
        return False
    
    return True

# Load configuration
config_path = project_root / 'config' / 'model_config.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

if not validate_config(config):
    raise ValueError("Configuration validation failed")
print("Configuration validated successfully!")

## 3. Run Simulation

In [ ]:
# Initialize simulation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

sim = EarthSystemSimulation(str(config_path), device)
print("Simulation initialized!")

# Run simulation with progress tracking
def run_simulation_with_progress(sim, num_steps, save_frequency):
    print("Running simulation...")
    try:
        trajectory = sim.run_simulation(
            num_steps=num_steps,
            save_frequency=save_frequency
        )
        print("Simulation complete!")
        return trajectory
    except Exception as e:
        print(f"Error during simulation: {str(e)}")
        raise

# Run simulation
trajectory = run_simulation_with_progress(sim, num_steps=50, save_frequency=5)

## 4. Analyze Results

In [ ]:
# Plot results
fig, axes = plt.subplots(3, 1, figsize=(12, 12))

# Physical system - mean temperature
temp_mean = np.mean(trajectory['physical'][..., 1], axis=(2, 3))  # Temperature channel
axes[0].plot(temp_mean, 'r-', label='Mean Temperature')
axes[0].set_title('Physical System')
axes[0].grid(True)
axes[0].legend()

# Biosphere - mean vegetation
veg_mean = np.mean(trajectory['biosphere'][..., 0], axis=0)  # Vegetation channel
axes[1].plot(veg_mean, 'g-', label='Mean Vegetation')
axes[1].set_title('Biosphere System')
axes[1].grid(True)
axes[1].legend()

# Geosphere - mean elevation
elev_mean = np.mean(trajectory['geosphere'][..., 0], axis=0)  # Elevation channel
axes[2].plot(elev_mean, 'b-', label='Mean Elevation')
axes[2].set_title('Geosphere System')
axes[2].grid(True)
axes[2].legend()

plt.tight_layout()
plt.show()

## 5. Save Results

In [ ]:
# Save results to file
output_file = 'simulation_results.npz'
np.savez(
    output_file,
    physical_states=np.array(trajectory['physical']),
    biosphere_states=np.array(trajectory['biosphere']),
    geosphere_states=np.array(trajectory['geosphere']),
    times=trajectory['times']
)
print(f"Results saved to: {output_file}")